In [30]:
# used to load text
from langchain.document_loaders import WebBaseLoader

# used to create the retriever
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

# used to create the retrieval tool
from langchain.agents import tool

# used to create the memory
from langchain.memory import ConversationBufferMemory

# used to create the prompt template
from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
from langchain.schema import SystemMessage
from langchain.prompts import MessagesPlaceholder

# used to create the agent executor
from langchain.chat_models import ChatOpenAI
from langchain.agents import AgentExecutor

In [34]:
import openai
openai.api_key = #insert api key here


# This is needed for both the memory and the prompt
openai.api_key =  memory_key = "history"

## The Retriever

In [35]:
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
data = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(data)

Created a chunk of size 2731, which is longer than the specified 1000
Created a chunk of size 1538, which is longer than the specified 1000
Created a chunk of size 1380, which is longer than the specified 1000
Created a chunk of size 2352, which is longer than the specified 1000
Created a chunk of size 1953, which is longer than the specified 1000
Created a chunk of size 1067, which is longer than the specified 1000
Created a chunk of size 1475, which is longer than the specified 1000
Created a chunk of size 2881, which is longer than the specified 1000
Created a chunk of size 1980, which is longer than the specified 1000
Created a chunk of size 4145, which is longer than the specified 1000
Created a chunk of size 2528, which is longer than the specified 1000


In [40]:
embeddings = OpenAIEmbeddings(openai_api_key=Insert API Key)


## The Retriever Tool

In [ ]:
db = FAISS.from_documents(texts, embeddings)
retriever = db.as_retriever()
@tool
def tool(query):
    "Searches and returns documents regarding the llm powered autonomous agents blog"
    docs = retriever.get_relevant_documents(query)
    return docs

tools = [tool]

## The Memory

In [42]:
memory = ConversationBufferMemory(memory_key=memory_key, return_messages=True)

## The Prompt Template



In [44]:
system_message = SystemMessage(
        content=(
            "Do your best to answer the questions. "
            "Feel free to use any tools available to look up "
            "relevant information, only if neccessary"
        )
)
prompt = OpenAIFunctionsAgent.create_prompt(
        system_message=system_message,
        extra_prompt_messages=[MessagesPlaceholder(variable_name=memory_key)]
    )

## The agent

In [ ]:
#llm = ChatOpenAI(temperature = 0, openai_api_key=openai_api_key)
llm = ChatOpenAI(temperature = 0, openai_api_key=API Key here)
agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)

## Agent Executer

In [ ]:
agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory, verbose=True)